In [ ]:
#hide
#default_exp core
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# covid19.data

This is a module that understands how to parse the new covid19 data schema from the [Johns Hopkins CSSE project](https://github.com/CSSEGISandData/COVID-19). As of 3/23, the schema of the dataset changed, and the overall time series data only reports aggregated country statistics. The daily data remains, but needs to be parsed into a format where we can track trends on a state-by-state basis in the US and a province-by-province basis in Canada.

In [ ]:
CONFIRMED_URI = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
DEATHS_URI = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"

In [ ]:
import pandas as pd

The following is the aggregated data, which is __mostly__ a country by country dataset. However, it is not consistent as there are provinces for Canada, but no individual states for the USA. Furthermore there are strange fields like "Diamond Princess" which presumably is just the data for that cruise ship. 

In [ ]:
confirmed_df = pd.read_csv(CONFIRMED_URI)
confirmed_df.head(10)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,21,22,22,22,24,24,40,40,74,84
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,51,55,59,64,70,76,89,104,123,146
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,54,60,74,87,90,139,201,230,264,302
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,2,39,39,53,75,88,113,133,164,188
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,0,0,0,0,1,2,2,3,3,3
5,NaN,Antigua and Barbuda,17.0608,-61.7964,0,0,0,0,0,0,...,1,1,1,1,1,1,1,3,3,3
6,NaN,Argentina,-38.4161,-63.6167,0,0,0,0,0,0,...,56,68,79,97,128,158,266,301,387,387
7,NaN,Armenia,40.0691,45.0382,0,0,0,0,0,0,...,52,78,84,115,136,160,194,235,249,265
8,Australian Capital Territory,Australia,-35.4735,149.0124,0,0,0,0,0,0,...,2,2,3,4,6,9,19,32,39,39
9,New South Wales,Australia,-33.8688,151.2093,0,0,0,0,3,4,...,171,210,267,307,353,436,669,669,818,1029


As we examine the daily reports, the individual day's data is constructed based on a filename with a `MM-DD-YYYY.csv` format, e.g., `03-24-2020.csv`. Let's define a function that reads the daily reports for a specific day.

In [ ]:
# export 
import datetime as dt

DAILY_REPORT_BASE_URI = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/"

def read_daily_report(date):
    """Read the daily report for date"""
    assert(isinstance(date, dt.date)), "parameter date must be a date object"
    uri = DAILY_REPORT_BASE_URI + date.strftime("%m-%d-%Y.csv")
    df = pd.read_csv(uri)
    df = df.rename(columns={"Province/State": "Province_State", "Country/Region": "Country_Region"})
    df.set_index(['Province_State', 'Country_Region'])
    return df

The daily report has several fields that are interesting. Let's compute a dataframe that contains the data for all the `Province_State` records in the US. We begin by reading the daily report into a dataframe and looking at its structure.

In [ ]:
df = read_daily_report(dt.date(2020,3,24))
df.head(5)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001.0,Abbeville,South Carolina,US,2020-03-24 23:37:31,34.223334,-82.461707,1,0,0,0,"Abbeville, South Carolina, US"
1,22001.0,Acadia,Louisiana,US,2020-03-24 23:37:31,30.295065,-92.414197,2,0,0,0,"Acadia, Louisiana, US"
2,51001.0,Accomack,Virginia,US,2020-03-24 23:37:31,37.767072,-75.632346,1,0,0,0,"Accomack, Virginia, US"
3,16001.0,Ada,Idaho,US,2020-03-24 23:37:31,43.452658,-116.241552,19,0,0,0,"Ada, Idaho, US"
4,19001.0,Adair,Iowa,US,2020-03-24 23:37:31,41.330756,-94.471059,1,0,0,0,"Adair, Iowa, US"


Next we want to do a pandas group by operation over `Province_State` for all `Country_Region == 'US'`. We will use the pandas [groupby](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) function to do this:

In [ ]:
states_df = df.loc[df['Country_Region'] == 'US'].groupby("Province_State").sum()
states_df.head(5)

,FIPS,Lat,Long_,Confirmed,Deaths,Recovered,Active
Province_State,,,,,,,
Alabama,71489.0,2203.246784,-5809.578199,242,0,0,0
Alaska,62340.0,1747.579877,-4229.319334,34,0,0,0
American Samoa,60000.0,-14.271000,-170.132000,0,0,0,0
Arizona,60208.0,505.138555,-1671.948482,326,5,0,0
Arkansas,380625.0,2618.391704,-6932.548370,219,2,0,0


Note that these are regions in the US, not just the states so we have rows for American Samoa, Puerto Rico, District of Columbia among others.

In [ ]:
len(states_df)

60

However, this is data for just a single point in time, i.e., 03-24-2020. We want the time series data, so we'll need to go back through time to compute this data. Let's create a function that will load a local dataframe that contains the cached results for all states.

In [ ]:
import urllib

def read_historical_data(start_date):
    """Read all of the historical data starting at start_date"""
    def daterange(start, end):
        for day in range(int((end-start).days)+1):
            yield start + dt.timedelta(day)
    
    result = []
    for day in daterange(start_date, dt.date.today()):
        try:
            df = read_daily_report(day)
            result.append((day, df))
        except urllib.error.HTTPError as err:
            # Today's data may not be available
            if err.code == 404 and day == dt.date.today():
                return result
    return result

Compute a dictionary from the historical data

In [ ]:
historical_data = read_historical_data(dt.date(2020,3,7))

Let's do the computation using just two dataframes

In [ ]:
d, df2 = historical_data[1]
print(d)
df2.head(5)

2020-03-08


,Province_State,Country_Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude
0,Hubei,Mainland China,2020-03-08T14:43:03,67707,2986,45235,30.9756,112.2707
1,NaN,Italy,2020-03-08T18:03:04,7375,366,622,43.0000,12.0000
2,NaN,South Korea,2020-03-08T12:53:03,7314,50,118,36.0000,128.0000
3,NaN,Iran,2020-03-08T11:03:30,6566,194,2134,32.0000,53.0000
4,Guangdong,Mainland China,2020-03-08T14:43:03,1352,7,1256,23.3417,113.4244


In [ ]:
d, df1 = historical_data[len(historical_data)-1]
print(d)
df1.loc[(df1["Province_State"]=="South Carolina") & (df1["Country_Region"]=="US")].head(5)

2020-03-25


,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001.0,Abbeville,South Carolina,US,2020-03-25 23:33:19,34.223334,-82.461707,3,0,0,0,"Abbeville, South Carolina, US"
21,45003.0,Aiken,South Carolina,US,2020-03-25 23:33:19,33.543380,-81.636454,2,0,0,0,"Aiken, South Carolina, US"
51,45005.0,Allendale,South Carolina,US,2020-03-25 23:33:19,32.988374,-81.353211,0,0,0,0,"Allendale, South Carolina, US"
61,45007.0,Anderson,South Carolina,US,2020-03-25 23:33:19,34.518281,-82.639595,19,0,0,0,"Anderson, South Carolina, US"
124,45009.0,Bamberg,South Carolina,US,2020-03-25 23:33:19,33.219276,-81.056600,0,0,0,0,"Bamberg, South Carolina, US"


Note that the schema changed on `03-23-2020` from:

`Province/State, Country/Region, Last Update, Confirmed, Deaths, Recovered, Latitude, Longitude`

to:

`FIPS, Admin2, Province_State, Country_Region, Lat, Long_, Confirmed, Deaths, Recovered, Active, Combined_Key`

We will use the latter schema and convert data from the older schema to the newer schema.

In [ ]:
df1 = df1.rename(columns={"Confirmed": dt.date(2020,3,25)})
df1.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,2020-03-25,Deaths,Recovered,Active,Combined_Key
0,45001.0,Abbeville,South Carolina,US,2020-03-25 23:33:19,34.223334,-82.461707,3,0,0,0,"Abbeville, South Carolina, US"
1,22001.0,Acadia,Louisiana,US,2020-03-25 23:33:19,30.295065,-92.414197,2,0,0,0,"Acadia, Louisiana, US"
2,51001.0,Accomack,Virginia,US,2020-03-25 23:33:19,37.767072,-75.632346,2,0,0,0,"Accomack, Virginia, US"
3,16001.0,Ada,Idaho,US,2020-03-25 23:33:19,43.452658,-116.241552,24,0,0,0,"Ada, Idaho, US"
4,19001.0,Adair,Iowa,US,2020-03-25 23:33:19,41.330756,-94.471059,1,0,0,0,"Adair, Iowa, US"


There is no state-wide aggregate data in earlier reports. So we have a couple of different options here:

1. We can create a manual mapping 
1. We can create an autoamtic mapping based on some kind of regex (but even a cursory inspection says this will be hard, i.e., "King County" vs. "King"
1. We can create a state-wide mapping again based on some kind of mapping

In [ ]:
df2.loc[(df2["Country_Region"]=="US")]

,Province_State,Country_Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude
45,"King County, WA",US,2020-03-08T20:23:09,83,17,1,47.5480,-121.9836
46,"Westchester County, NY",US,2020-03-08T18:03:07,83,0,0,41.1220,-73.7949
57,Unassigned Location (From Diamond Princess),US,2020-03-02T19:53:03,45,0,0,35.4437,139.6380
62,"Santa Clara County, CA",US,2020-03-08T21:23:03,38,0,1,37.3541,-121.9552
67,"Snohomish County, WA",US,2020-03-08T21:43:03,31,1,0,48.0330,-121.8339
...,...,...,...,...,...,...,...,...
246,"Yolo County, CA",US,2020-03-06T20:13:14,1,0,0,38.7646,-121.9018
251,"Lackland, TX (From Diamond Princess)",US,2020-02-24T23:33:02,0,0,0,29.3829,-98.6134
252,"Montgomery County, TX",US,2020-03-07T19:53:02,0,0,0,30.3213,-95.4778
253,"Omaha, NE (From Diamond Princess)",US,2020-02-24T23:33:02,0,0,0,41.2545,-95.9758


Let's try a simple regex against `Province_State` to see what that reveals

In [ ]:
import re

str = "King County, WA"
res = re.match("(?P<county>.*?)\s+County,\s+(?P<state>.*)", str)
county, state = res.group(1), res.group(2)
print(county, state)

King WA


Need a table of state abbreviations

In [ ]:
state_abbreviations = pd.read_csv("./data/state_abbreviations.csv", index_col="Abbreviation")
state_abbreviations.head(5)

,State
Abbreviation,
AL,Alabama
AK,Alaska
AZ,Arizona
AR,Arkansas
CA,California


Now let's lookup a state based on its abbreviation

In [ ]:
state_abbreviations.loc['NY'].iloc[0]

'New York'

This is getting closer, but I really want to apply a lambda function across each row in this result set

In [ ]:
def extract_state(province_state):
    if not isinstance(province_state, type(str)):
        return ""
    res = re.match("(.*?)\s+County,\s+(\S*)", province_state)
    if res == None:
        return province_state
    county, state = res.group(1), res.group(2)
    result = state_abbreviations.loc[state].iloc[0]
    return result

In [ ]:
def extract_county(province_state):
    if not isinstance(province_state, type(str)):
        return ""
    res = re.match("(.*?)\s+County,\s+(\S*)", province_state)
    if res == None:
        return ""
    return res.group(1)

In [ ]:
df2["Province_State"]

0                                     Hubei
1                                       NaN
2                                       NaN
3                                       NaN
4                                 Guangdong
                       ...                 
250                      Northern Territory
251    Lackland, TX (From Diamond Princess)
252                   Montgomery County, TX
253       Omaha, NE (From Diamond Princess)
254      Travis, CA (From Diamond Princess)
Name: Province_State, Length: 255, dtype: object

In [ ]:
df2["Admin2"] = df2["Province_State"].apply(extract_county)
df2["Province_State"] = df2["Province_State"].apply(extract_state)
df2.loc[df2["Country_Region"]=="US"]

,Province_State,Country_Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude,Admin2,Admin,Province_State2
45,Washington,US,2020-03-08T20:23:09,83,17,1,47.5480,-121.9836,King,King,Washington
46,New York,US,2020-03-08T18:03:07,83,0,0,41.1220,-73.7949,Westchester,Westchester,New York
57,Unassigned Location (From Diamond Princess),US,2020-03-02T19:53:03,45,0,0,35.4437,139.6380,,,Unassigned Location (From Diamond Princess)
62,California,US,2020-03-08T21:23:03,38,0,1,37.3541,-121.9552,Santa Clara,Santa Clara,California
67,Washington,US,2020-03-08T21:43:03,31,1,0,48.0330,-121.8339,Snohomish,Snohomish,Washington
...,...,...,...,...,...,...,...,...,...,...,...
246,California,US,2020-03-06T20:13:14,1,0,0,38.7646,-121.9018,Yolo,Yolo,California
251,"Lackland, TX (From Diamond Princess)",US,2020-02-24T23:33:02,0,0,0,29.3829,-98.6134,,,"Lackland, TX (From Diamond Princess)"
252,Texas,US,2020-03-07T19:53:02,0,0,0,30.3213,-95.4778,Montgomery,Montgomery,Texas
253,"Omaha, NE (From Diamond Princess)",US,2020-02-24T23:33:02,0,0,0,41.2545,-95.9758,,,"Omaha, NE (From Diamond Princess)"


Now let's update the `read_daily_report` function to perform transform on old schema files:

In [ ]:
# export 
import pandas as pd
import datetime as dt

DAILY_REPORT_BASE_URI = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/"

def read_daily_report(date):
    """Read the daily report for date"""
    assert(isinstance(date, dt.date)), "parameter date must be a date object"
    uri = DAILY_REPORT_BASE_URI + date.strftime("%m-%d-%Y.csv")
    df = pd.read_csv(uri)
    
    # Adjust for old schema
    if date < dt.date(2020, 3, 22):
        df = df.rename(columns={"Province/State": "Province_State", "Country/Region": "Country_Region"})
        df["Admin2"] = df["Province_State"].apply(extract_county)
        df["Province_State"] = df["Province_State"].apply(extract_state)
        
    return df

In [ ]:
historical_data = read_historical_data(dt.date(2020,3,7))

In [ ]:
d, df = historical_data[0]
df.loc[df["Country_Region"]=="US"]

,Province_State,Country_Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude,Admin2
48,Washington,US,2020-03-07T20:23:03,71,15,1,47.5480,-121.9836,King
50,New York,US,2020-03-07T18:23:05,57,0,0,41.1220,-73.7949,Westchester
56,Unassigned Location (From Diamond Princess),US,2020-03-02T19:53:03,45,0,0,35.4437,139.6380,
59,California,US,2020-03-07T23:43:03,32,0,1,37.3541,-121.9552,Santa Clara
62,Washington,US,2020-03-07T20:23:03,27,1,0,48.0330,-121.8339,Snohomish
...,...,...,...,...,...,...,...,...,...
216,California,US,2020-03-06T20:13:14,1,0,0,38.7646,-121.9018,Yolo
221,"Lackland, TX (From Diamond Princess)",US,2020-02-24T23:33:02,0,0,0,29.3829,-98.6134,
222,Texas,US,2020-03-07T19:53:02,0,0,0,30.3213,-95.4778,Montgomery
223,"Omaha, NE (From Diamond Princess)",US,2020-02-24T23:33:02,0,0,0,41.2545,-95.9758,


Now let's take an old schema dataframe `df2` and a new schema dataframe `df1` and join

In [ ]:
df1 = df1.rename(columns={"Confirmed": dt.date(2020,3,25)})
df1.head(5)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,2020-03-25,Deaths,Recovered,Active,Combined_Key
0,45001.0,Abbeville,South Carolina,US,2020-03-25 23:33:19,34.223334,-82.461707,3,0,0,0,"Abbeville, South Carolina, US"
1,22001.0,Acadia,Louisiana,US,2020-03-25 23:33:19,30.295065,-92.414197,2,0,0,0,"Acadia, Louisiana, US"
2,51001.0,Accomack,Virginia,US,2020-03-25 23:33:19,37.767072,-75.632346,2,0,0,0,"Accomack, Virginia, US"
3,16001.0,Ada,Idaho,US,2020-03-25 23:33:19,43.452658,-116.241552,24,0,0,0,"Ada, Idaho, US"
4,19001.0,Adair,Iowa,US,2020-03-25 23:33:19,41.330756,-94.471059,1,0,0,0,"Adair, Iowa, US"


In [ ]:
df2 = df2.rename(columns={dt.date(2020,3,8): 'Confirmed'})
df2.head(5)

,Province_State,Country_Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude,Admin2,Admin,Province_State2
0,Hubei,Mainland China,2020-03-08T14:43:03,67707,2986,45235,30.9756,112.2707,,,Hubei
1,,Italy,2020-03-08T18:03:04,7375,366,622,43.0000,12.0000,,,
2,,South Korea,2020-03-08T12:53:03,7314,50,118,36.0000,128.0000,,,
3,,Iran,2020-03-08T11:03:30,6566,194,2134,32.0000,53.0000,,,
4,Guangdong,Mainland China,2020-03-08T14:43:03,1352,7,1256,23.3417,113.4244,,,Guangdong


Now we want to join `df1` and `df2` using `Admin2`, `Province_State`, and `Country_Region`, extract the `Confirmed` column and append it as a new column with the date of that dataframe

In [ ]:
print(len(df1), len(df2))

3420 255


In [ ]:
df = df1[["FIPS", "Admin2", "Province_State", "Country_Region", "Lat", "Long_", dt.date(2020,3,25)]].merge(df2[["Admin2", "Province_State", "Country_Region", "Confirmed"]], on=["Admin2", "Province_State", "Country_Region"], how="outer").fillna(0, downcast='infer')
df.head(10)

,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,2020-03-25,Confirmed
0,45001,Abbeville,South Carolina,US,34.223334,-82.461707,3,0
1,22001,Acadia,Louisiana,US,30.295065,-92.414197,2,0
2,51001,Accomack,Virginia,US,37.767072,-75.632346,2,0
3,16001,Ada,Idaho,US,43.452658,-116.241552,24,0
4,19001,Adair,Iowa,US,41.330756,-94.471059,1,0
5,21001,Adair,Kentucky,US,37.104598,-85.281297,0,0
6,29001,Adair,Missouri,US,40.190586,-92.600782,1,0
7,40001,Adair,Oklahoma,US,35.884942,-94.658593,2,0
8,8001,Adams,Colorado,US,39.874321,-104.336258,27,0
9,16003,Adams,Idaho,US,44.893336,-116.454525,0,0


Now let's write a function that will do the merge from most recent to oldest

In [ ]:
initialized = False
for d, df in reversed(historical_data):
    if not initialized:
        cdf = df[["FIPS", "Admin2", "Province_State", "Country_Region", "Lat", "Long_", "Confirmed"]].fillna(value={"Admin2": "", "Province_State": ""}, downcast="infer")
        ddf = df[["FIPS", "Admin2", "Province_State", "Country_Region", "Lat", "Long_", "Deaths"]].fillna(value={"Admin2": "", "Province_State": ""}, downcast="infer")
        initialized = True
    else:
        df = df.fillna(value={"Admin2": "", "Province_State": ""}, downcast="infer")
        cdf = cdf.merge(df[["Admin2", "Province_State", "Country_Region", "Confirmed"]], on=["Admin2", "Province_State", "Country_Region"], how="outer")
        ddf = ddf.merge(df[["Admin2", "Province_State", "Country_Region", "Deaths"]], on=["Admin2", "Province_State", "Country_Region"], how="outer")

    # Rename the merged columns
    cdf = cdf.rename(columns={"Confirmed": d})
    ddf = ddf.rename(columns={"Deaths": d})

In [ ]:
cdf.loc[cdf["Country_Region"]=="Canada"]

,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,2020-03-26,2020-03-25,2020-03-24,2020-03-23,...,2020-03-16,2020-03-15,2020-03-14,2020-03-13,2020-03-12,2020-03-11,2020-03-10,2020-03-09,2020-03-08,2020-03-07
3167,NaN,,Alberta,Canada,53.9333,-116.5765,486.0,358.0,359.0,301.0,...,56.0,39.0,29.0,29.0,19.0,19.0,7.0,7.0,NaN,NaN
3174,NaN,,British Columbia,Canada,53.7267,-127.6476,725.0,617.0,617.0,472.0,...,103.0,73.0,64.0,64.0,46.0,39.0,32.0,32.0,27.0,21.0
3179,NaN,,Diamond Princess,Canada,0.0000,0.0000,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3187,NaN,,Grand Princess,Canada,0.0000,0.0000,13.0,13.0,13.0,13.0,...,2.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
3209,NaN,,Manitoba,Canada,53.7609,-98.8139,36.0,35.0,21.0,20.0,...,7.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
3213,NaN,,New Brunswick,Canada,46.5653,-66.4619,33.0,18.0,18.0,17.0,...,6.0,2.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
3216,NaN,,Newfoundland and Labrador,Canada,53.1355,-57.6604,82.0,35.0,35.0,24.0,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3220,NaN,,Northwest Territories,Canada,64.8255,-124.8457,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3221,NaN,,Nova Scotia,Canada,44.6820,-63.7443,73.0,68.0,51.0,41.0,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3222,NaN,,Ontario,Canada,51.2538,-85.3232,858.0,688.0,588.0,503.0,...,177.0,104.0,79.0,74.0,42.0,41.0,36.0,34.0,NaN,NaN


Save the data for now

In [ ]:
cdf.to_csv("./data/confirmed_timeseries.csv")
ddf.to_csv("./data/death_timeseries.csv")

## Creating an API for managing the data

The API needs to do a few things:

1. Look for local cached copy of processed data
1. If not present, load data from the beginning of time
1. If present, load from the most recent date in cache
1. Process data
1. Cache results

Experiments with loading data and finding most recent date in columns

In [ ]:
#export 

import pandas as pd
import datetime as dt

def try_parse(date):
    """Try and parse a string into a date. Returns a tuple of (True/False, parsed date)"""
    try:
        result = dt.datetime.strptime(date, "%Y-%m-%d")
        return (True, result)
    except ValueError:
        return (False, None)
    
def get_dates(columns):
    """Construct a list from an existing list that has date and non-date values in it"""
    if len(columns) == 0:
        raise ValueError("columns must have length > 0")
        
    result = []
    for column in columns:
        is_date, date = try_parse(column)
        if is_date:
            result.append(date.date())
    return result

def find_max(dates):
    """Find maximum date value in list"""
    if len(dates) == 0:
        raise ValueError("dates must have length > 0")
        
    max = dates[0]
    for i in range(1, len(dates)):
        if dates[i] > max:
            max = dates[i]
    return max

Test the code above ... really need to turn this into a proper test once I figure out how to do this

In [ ]:
df = pd.read_csv("./data/confirmed_timeseries.csv")
find_max(get_dates(df.columns))

FileNotFoundError: [Errno 2] File ./data/confirmed_timeseries.csv does not exist: './data/confirmed_timeseries.csv'

Test variation of the algorithm that will start with oldest daily update file and append data until we get to today. First begin with the code that will read the daily report and adjust between new vs. old schema.

In [ ]:
# export 
import re
import datetime as dt

DAILY_REPORT_BASE_URI = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/"

state_abbreviations = pd.read_csv("./data/state_abbreviations.csv", index_col="Abbreviation")

def extract_state(province_state):
    province_state = str(province_state)
    if province_state == "":
        return province_state
    res = re.match("(.*?)\s+County,\s+(\S*)", province_state)
    if res == None:
        res = re.match("(.*?),\s+(\S*)", province_state)
        if res == None:
            return province_state
    county, state = res.group(1), res.group(2)
    if state in state_abbreviations.index:
        return state_abbreviations.loc[state].iloc[0]
    else:
        return state

def extract_county(province_state):
    province_state = str(province_state)
    if province_state == "":
        return province_state
    res = re.match("(.*?)(\s+County)?,\s+(\S*)", province_state)
    if res == None:
        res = re.match("(.*?),\s+(\S*)", province_state)
        if res == None:
            return province_state
        else:
            return res.group(1)
    return res.group(1)

def read_daily_report(date):
    """Read the daily report for date"""
    assert(isinstance(date, dt.date)), "parameter date must be a date object"
    uri = DAILY_REPORT_BASE_URI + date.strftime("%m-%d-%Y.csv")
    df = pd.read_csv(uri)
    
    # Adjust for old schema
    if date < dt.date(2020, 3, 22):
        df = df.rename(columns={"Province/State": "Province_State", "Country/Region": "Country_Region"})
        df["Admin2"] = df["Province_State"].apply(extract_county)
        df["Province_State"] = df["Province_State"].apply(extract_state)
        df["FIPS"] = 0
        df["Lat"] = 0.0
        df["Long_"] = 0.0
        
    return df

Next, let's add the function that will read each historical daily file and append to a list containing tuples of (date, df)

In [ ]:
# export
import urllib

EARLIEST_DATE = dt.date(2020, 1, 22)

def read_historical_data(start_date, end_date):
    """Read all of the historical data starting at start_date"""
    def daterange(start, end):
        for day in range(int((end-start).days)+1):
            yield start + dt.timedelta(day)
    
    result = []
    for day in daterange(start_date, end_date):
        try:
            df = read_daily_report(day)
            result.append((day, df))
        except urllib.error.HTTPError as err:
            # Today's data may not be available
            if err.code == 404 and day == dt.date.today():
                return result
    return result

Now let's read the data 

In [ ]:
historical_data = read_historical_data(EARLIEST_DATE, dt.date(2020, 3, 27))
len(historical_data)

66

Now let's try running the algorithm from the oldest date to the newest date

In [ ]:
initialized = False
for d, df in historical_data:
    if not initialized:
        cdf = df[["FIPS", "Admin2", "Province_State", "Country_Region", "Lat", "Long_", "Confirmed"]].fillna(value={"Admin2": ""}, downcast="infer")
        ddf = df[["FIPS", "Admin2", "Province_State", "Country_Region", "Lat", "Long_", "Deaths"]].fillna(value={"Admin2": ""}, downcast="infer")
        initialized = True
    else:
        df = df.fillna(value={"Admin2": "", "Province_State": ""}, downcast="infer")
        cdf = cdf.merge(df[["FIPS", "Admin2", "Province_State", "Country_Region", "Confirmed", "Lat", "Long_"]], on=["FIPS", "Admin2", "Province_State", "Country_Region", "Lat", "Long_"], how="outer")
        ddf = ddf.merge(df[["FIPS", "Admin2", "Province_State", "Country_Region", "Deaths", "Lat", "Long_"]], on=["FIPS", "Admin2", "Province_State", "Country_Region", "Lat", "Long_"], how="outer")

    # Rename the merged columns
    cdf = cdf.rename(columns={"Confirmed": d})
    ddf = ddf.rename(columns={"Deaths": d})

Write out the time series to disk so I can inspect it using Excel you know I meant Excel.

In [ ]:
cdf.tail(10)
cdf.to_csv("./data/confirmed_timeseries.csv")

Bringing everything together into a single function that initializes the data

In [ ]:
#export

import os
import datetime as dt
import pandas as pd

DAILY_REPORT_BASE_URI = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/"
CACHED_CONFIRMED = "./data/confirmed_timeseries.csv"
CACHED_DEATHS = "./data/death_timeseries.csv"
EARLIEST_DATE = dt.date(2020, 1, 22)

confirmed_df = None
deaths_df = None

def init(most_recent_date):
    """Initialize API from most_recent_date to earliest known date while taking advantage of local caching"""
    
    global confirmed_df, deaths_df
    if most_recent_date == None:
        most_recent_date = dt.date.today()
    
    # I'm not in the mood to handle partial failures
    cache_ok = False
    start_date = EARLIEST_DATE
    if os.path.isfile(CACHED_CONFIRMED) and os.path.isfile(CACHED_DEATHS):
        confirmed_df = pd.read_csv(CACHED_CONFIRMED)
        deaths_df = pd.read_csv(CACHED_DEATHS)

        # Find the most recent date based on column labels
        confirmed_most_recent_date = find_max(get_dates(confirmed_df))
        deaths_most_recent_date = find_max(get_dates(deaths_df))

        if confirmed_most_recent_date == deaths_most_recent_date:
            cache_ok = True
            start_date = confirmed_most_recent_date

    # Read historical data from start_date to today
    historical_data = read_historical_data(start_date, most_recent_date)
    
    initialized = False
    for d, df in historical_data:
        if not initialized:
            if not cache_ok:
                confirmed_df = df[["FIPS", "Admin2", "Province_State", "Country_Region", "Lat", "Long_", "Confirmed"]].fillna(value={"Admin2": ""}, downcast="infer")
                deaths_df = df[["FIPS", "Admin2", "Province_State", "Country_Region", "Lat", "Long_", "Deaths"]].fillna(value={"Admin2": ""}, downcast="infer")
            initialized = True
        else:
            df = df.fillna(value={"Admin2": "", "Province_State": ""}, downcast="infer")
            confirmed_df = confirmed_df.merge(df[["FIPS", "Admin2", "Province_State", "Country_Region", "Lat", "Long_", "Confirmed"]], on=["FIPS", "Admin2", "Province_State", "Country_Region", "Lat", "Long_"], how="outer")
            deaths_df = deaths_df.merge(df[["FIPS", "Admin2", "Province_State", "Country_Region", "Lat", "Long_", "Deaths"]], on=["FIPS", "Admin2", "Province_State", "Country_Region", "Lat", "Long_"], how="outer")

        # Rename the merged columns
        confirmed_df = confirmed_df.rename(columns={"Confirmed": str(d)})
        deaths_df = deaths_df.rename(columns={"Deaths": str(d)})

    # Write back out to cache
    confirmed_df.to_csv(CACHED_CONFIRMED, index=False)
    deaths_df.to_csv(CACHED_DEATHS, index=False)

In [ ]:
init(dt.date.today())

## Now let's write some APIs that will retrieve the data from the dataset

`get_data(country, region, sub_region)`. If given just `country`, the API will return the aggregated time series for the entire country. If given `country, region` it will return the aggregated time series for all of the sub regions within `country, region`. If given all three, `country, region, sub_region` it will return the time series for that tuple. I think what we need to do is use the pandas [groupby](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) API

The APIs will not return aggregated data, but instead will return just the dataset based on the query and its up to you to `sum()` over rows if that's what you want.

In [ ]:
dates = [str(d) for d in get_dates(confirmed_df.columns)]
res = confirmed_df.loc[(confirmed_df["Country_Region"]=="US") & (confirmed_df["Province_State"]=="Washington") & (confirmed_df["Admin2"] == "King"), dates].sum()
res

Candidate APIs:

Concepts: `country`, `state`, `region`

- `get_all_countries()` returns aggregated data for each country
- `get_all_regions_within_state(country, state)` if state is None then returns all regions for that country. if state has a value, returns all regions for that state.
- `get_region(country, state, region)` returns a time series for just that region (or should it be a dataframe with just that row ...?


Compute the aggregated data for all countries

In [ ]:
confirmed_df.groupby(["Country_Region"]).sum()[["Lat", "Long_"] + dates]

Return all regions within the US

In [ ]:
confirmed_df.loc[(confirmed_df["Country_Region"]=="US")]

Return all regions in US grouped by region

In [ ]:
confirmed_df.loc[(confirmed_df["Country_Region"]=="US")].groupby(["Province_State"]).sum()[["Lat", "Long_"] + dates]

Return all regions within Washington State in US

In [ ]:
confirmed_df.loc[(confirmed_df["Country_Region"] == "US") & (confirmed_df["Province_State"] == "Washington")][["Admin2", "Province_State", "Country_Region", "Lat", "Long_"]+dates]

Return specific region

In [ ]:
confirmed_df.loc[(confirmed_df["Country_Region"] == "US") & (confirmed_df["Province_State"] == "Washington") & (confirmed_df["Admin2"] == "King")][["Admin2", "Province_State", "Country_Region", "Lat", "Long_"]+dates]

Get the metadata for each `Country_Region`

In [ ]:
confirmed_df.drop_duplicates("Country_Region")[["Country_Region", "Lat", "Long_"]]

Let's get a list of all of the `Province_State` within a `Country_Region`

In [ ]:
confirmed_df.loc[confirmed_df["Country_Region"]=="US"]["Province_State"].unique()

Let's define an API that will retrieve either confirmed cases or deaths given `country`, `region`, `sub_region` as parameters:

In [ ]:
# export

def get_date_column_names(df):
    return [str(d) for d in get_dates(df.columns)]

def get_countries():
    columns = ["Lat", "Long_"] + get_date_column_names(confirmed_df)
    return (confirmed_df.groupby(["Country_Region"]).sum()[columns],
            deaths_df.groupby(["Country_Region"]).sum()[columns])

In [ ]:
get_countries()[0]

In [ ]:
# export

def get_country_states(country):
    columns = ["FIPS", "Lat", "Long_"] + get_date_column_names(confirmed_df)
    return (confirmed_df.loc[confirmed_df["Country_Region"] == country].groupby(["Province_State", "Country_Region"]).sum()[columns],
            deaths_df.loc[deaths_df["Country_Region"] == country].groupby(["Province_State", "Country_Region"]).sum()[columns])

In [ ]:
get_country_states("US")[0]

In [ ]:
# export

def get_country_state_regions(country, state):
    columns = ["Country_Region", "Province_State", "Admin2", "FIPS", "Lat", "Long_"] + get_date_column_names(confirmed_df)
    return (confirmed_df.loc[(confirmed_df["Country_Region"] == country) & (confirmed_df["Province_State"] == state)][columns],
            deaths_df.loc[(deaths_df["Country_Region"] == country) & (deaths_df["Province_State"] == state)][columns])

In [ ]:
get_country_state_regions("US", "New York")[0]

NameError: name 'get_date_column_names' is not defined

Define some APIs for getting names of countries, states, and regions for interactive applications

In [ ]:
# export

def get_country_names():
    return confirmed_df["Country_Region"].unique()

def get_state_names(country):
    return confirmed_df.loc[confirmed_df["Country_Region"]==country]["Province_State"].unique()

def get_region_names(country, state):
    return confirmed_df.loc[(confirmed_df["Country_Region"]==country)&(confirmed_df["Province_State"]==state)]["Admin2"].unique()

In [ ]:
get_region_names("US", "Washington")